In [1]:
@file:DependsOn("org.jetbrains.kotlinx:kotlin-deeplearning-api:0.3.0")

import org.jetbrains.kotlinx.dl.api.core.Sequential
import org.jetbrains.kotlinx.dl.api.core.activation.Activations
import org.jetbrains.kotlinx.dl.api.core.initializer.GlorotNormal
import org.jetbrains.kotlinx.dl.api.core.initializer.Zeros
import org.jetbrains.kotlinx.dl.api.core.layer.core.Dense
import org.jetbrains.kotlinx.dl.api.core.layer.core.Input
import org.jetbrains.kotlinx.dl.api.core.loss.Losses
import org.jetbrains.kotlinx.dl.api.core.metric.Metrics
import org.jetbrains.kotlinx.dl.api.core.optimizer.Adam
import org.jetbrains.kotlinx.dl.api.core.summary.logSummary
import org.jetbrains.kotlinx.dl.dataset.OnHeapDataset
import kotlin.random.Random

In [3]:
val SEED = 12L
val TEST_BATCH_SIZE = 1000
val EPOCHS = 100
val TRAINING_BATCH_SIZE = 100

In [4]:
val model = Sequential.of(
    Input(4),
    Dense(1, Activations.Linear, kernelInitializer = GlorotNormal(), biasInitializer = Zeros(), name = "dense_2")
)

This example shows how to do regression from scratch, starting from generated dataset, using simple Dense-based [model] with 1 neuron.
It includes:
 - dataset creation
 - dataset splitting
 - model compilation
 - model training
 - model evaluation
 - model weights printing

In [5]:
val rnd = Random(SEED)

In [7]:
val data = Array(10000) { doubleArrayOf(0.0, 0.0, 0.0, 0.0, 0.0) }

for (i in data.indices) {
    data[i][1] = 2 * (rnd.nextDouble() - 0.5)
    data[i][2] = 2 * (rnd.nextDouble() - 0.5)
    data[i][3] = 2 * (rnd.nextDouble() - 0.5)
    data[i][4] = 2 * (rnd.nextDouble() - 0.5)
    data[i][0] = data[i][1] - 2 * data[i][2] + 1.5 * data[i][3] - 0.95 * data[i][4] + rnd.nextDouble(0.1)
    // 1 * x1 - 2 * x2 + 1.5 * x3  - 0.95 * x4 +- 0.1
}

data.shuffle()

In [9]:
fun extractX(): Array<FloatArray> {
    val init: (index: Int) -> FloatArray = { index ->
        floatArrayOf(
            data[index][1].toFloat(),
            data[index][2].toFloat(),
            data[index][3].toFloat(),
            data[index][4].toFloat()
        )
    }
    return Array(data.size, init = init)
}

In [11]:
fun extractY(): FloatArray {
    val labels = FloatArray(data.size) { 0.0f }
        for (i in labels.indices) {
        labels[i] = data[i][0].toFloat()
    }
    return labels
}

In [12]:
val dataset = OnHeapDataset.create(
    ::extractX,
    ::extractY
)

In [13]:
val (train, test) = dataset.split(0.9)

In [14]:
model.compile(
    optimizer = Adam(),
    loss = Losses.MSE,
    metric = Metrics.MAE
)

In [ ]:
// temporary not support, https://github.com/JetBrains/KotlinDL/pull/251
model.logSummary()

In [15]:
model.fit(dataset = train, epochs = EPOCHS, batchSize = TRAINING_BATCH_SIZE)

org.jetbrains.kotlinx.dl.api.core.history.TrainingHistory@5eccd3b9

In [17]:
repeat(100) { id ->
    val xReal = test.getX(id)
    val yReal = test.getY(id)
    val yPred = model.predictSoftly(xReal)

    println("xReal: ${arrayOf(xReal).contentDeepToString()}, yReal: $yReal, yPred: ${yPred[0]}")
}

xReal: [[0.5539668, -0.52224463, 0.87776214, -0.38244128]], yReal: 3.3727727, yPred: 3.2869294
xReal: [[0.35120124, -0.67848516, -0.07302495, -0.70261604]], yReal: 2.2710063, yPred: 2.2638047
xReal: [[0.73524225, -0.14792243, 0.28574997, 0.18923226]], yReal: 1.3489411, yPred: 1.317679
xReal: [[0.77088237, 0.4149414, -0.1166994, -0.41026783]], yReal: 0.23058084, yPred: 0.23685409
xReal: [[-0.83985275, 0.48582548, -0.93762654, -0.1811578]], yReal: -3.006621, yPred: -2.95868
xReal: [[-0.9319623, 0.03033123, 0.040475365, 0.400423]], yReal: -1.3023514, yPred: -1.2610584
xReal: [[0.5114596, -0.07594006, -0.6268773, 0.5511299]], yReal: -0.80048424, yPred: -0.7563286
xReal: [[0.71694493, 0.38426498, 0.46299422, 0.89486367]], yReal: -0.12234519, yPred: -0.12916163
xReal: [[-0.7549203, -0.54401046, -0.23129436, 0.28655922]], yReal: -0.21595323, yPred: -0.27823496
xReal: [[-0.22682856, -0.5217256, -0.9592582, -0.25867578]], yReal: -0.3647471, yPred: -0.3660003
xReal: [[-0.16958275, -0.63749295, -

In [20]:
val mae = model.evaluate(dataset = test, batchSize = TEST_BATCH_SIZE).metrics[Metrics.MAE]
println("Weights: " + model.getLayer("dense_2").weights["dense_2_dense_kernel"].contentDeepToString())
println("Bias: " + model.getLayer("dense_2").weights["dense_2_dense_bias"].contentDeepToString())
println("MAE: $mae")

Weights: [[1.0002059], [-1.9237784], [1.4988353], [-0.9500553]]
Bias: [0.049204376]
MAE: 0.0431436188519001


In [22]:
repeat(100) { id ->
    val xReal = test.getX(id)
    val yReal = test.getY(id)

    val yPred = model.predictSoftly(xReal)

    println("xReal: ${xReal[0]}, yReal: $yReal, yPred: ${yPred[0]}")
}

xReal: 0.5539668, yReal: 3.3727727, yPred: 3.2869294
xReal: 0.35120124, yReal: 2.2710063, yPred: 2.2638047
xReal: 0.73524225, yReal: 1.3489411, yPred: 1.317679
xReal: 0.77088237, yReal: 0.23058084, yPred: 0.23685409
xReal: -0.83985275, yReal: -3.006621, yPred: -2.95868
xReal: -0.9319623, yReal: -1.3023514, yPred: -1.2610584
xReal: 0.5114596, yReal: -0.80048424, yPred: -0.7563286
xReal: 0.71694493, yReal: -0.12234519, yPred: -0.12916163
xReal: -0.7549203, yReal: -0.21595323, yPred: -0.27823496
xReal: -0.22682856, yReal: -0.3647471, yPred: -0.3660003
xReal: -0.16958275, yReal: 1.0733652, yPred: 1.0101429
xReal: 0.55107397, yReal: -1.7156777, yPred: -1.6024302
xReal: -0.7413579, yReal: -2.8416684, yPred: -2.7721114
xReal: 0.040725425, yReal: -1.2625089, yPred: -1.271391
xReal: -0.114923686, yReal: -2.1096516, yPred: -2.0675976
xReal: -0.02724215, yReal: 1.4294659, yPred: 1.4000248
xReal: -0.38675752, yReal: 0.0931997, yPred: 0.016450945
xReal: -0.48338443, yReal: -0.9821144, yPred: -0.992

In [23]:
model.close()